In [1]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy as db
from sqlalchemy import create_engine
import yaml

# Adding database conection

In [2]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
co_sa = create_engine(url_co)
etl_conn = create_engine(url_etl)

# loading tables

In [3]:
beneficiarios = pd.read_sql_table("beneficiario", co_sa)
cotizantes = pd.read_sql_table("cotizante", co_sa)
cot_ben = pd.read_sql_table("cotizante_beneficiario", co_sa)

In [4]:
beneficiarios.head()

,tipo_identificacion,id_beneficiario,parentesco,nombre,fecha_nacimiento,sexo,estado_civil,tipo_discapacidad
0,Cedula,1705221112240,Mama,Deidamia Bernal Tello,1954-10-08,F,Soltero,Ninguna
1,Tarjeta Identidad,1705221115600,Hija,Nazaret Dueñas Dueñas,1997-10-16,F,Soltero,Ninguna
2,Cedula,1705221116830,Compañera,Afra Acosta Heredia,1975-08-21,F,Union Libre,Ninguna
3,Cedula,1705221117960,Papa,Tomás Dueñas Delatorre,1967-07-24,M,Union Libre,Ninguna
4,Cedula,1705221123050,Hermana,Celeste Díez Calderón,1949-08-05,F,Casado,Ninguna


In [5]:
cotizantes.head()

,cedula,nombre,tipo_cotizante,direccion,estado_civil,sexo,fecha_nacimiento,nivel_escolaridad,estracto,proviene_otra_eps,salario_base,fecha_afiliacion,tipo_discapacidad,id_ips
0,932170522,Eleodora Tapia Lopez,Independiente,Calle 9 # 95-124,Viudo,F,1983-09-05,N/A,4,1,2726000,2007-07-20,Ninguna,IPS_55
1,934170522,Belisaria Crespo Jerez,Independiente,Calle 16 # 126-93,Union Libre,F,1990-12-23,Técnico,1,0,476000,2006-05-03,Ninguna,IPS_35
2,936170522,Gerardo Villanueva Pinto,Independiente,Transversal 117 # 84-85,Casado,M,1983-10-23,N/A,1,0,476000,2006-10-08,Ninguna,IPS_33
3,937170522,Bienvenido Quintana Moreno,Jubilado,Calle 99 # 81-12,Viudo,M,1987-02-16,Universitaria,2,1,976000,2006-03-20,Ninguna,IPS_19
4,941170522,Celestino Medrano Moreno,Independiente,Kra 64 # 127-59,Union Libre,M,1990-01-24,Universitaria,4,0,1726000,2007-11-14,Ninguna,IPS_48


In [16]:
cot_ben

,cotizante,beneficiario
0,1170522,1705221112240
1,2170522,1705221115600
2,2170522,1705221116830
3,2170522,1705221117960
4,5170522,1705221123050
...,...,...
3311,998113543,1135438558010
3312,999113543,1135438560540
3313,999113543,1135438561640
3314,999113543,1135438562650


In [7]:
campos = ["tipo_documento","numero_identificacion","nombre","tipo_usuario","estado_civil","sexo","fecha_nacimiento","tipo_discapacidad","grupo_familiar"]

# Modificaciones cotizante

In [8]:
cotizantes.rename(columns={'cedula':'numero_identificacion'}, inplace=True)
cotizantes.drop(columns=['direccion','tipo_cotizante','nivel_escolaridad','estracto','proviene_otra_eps','salario_base','fecha_afiliacion','id_ips'], inplace=True)
cotizantes['tipo_documento'] = "cedula"
cotizantes['tipo_usuario'] = "cotizante"
cotizantes['grupo_familiar'] = cotizantes['numero_identificacion']
cotizantes.head()

,numero_identificacion,nombre,estado_civil,sexo,fecha_nacimiento,tipo_discapacidad,tipo_documento,tipo_usuario,grupo_familiar
0,932170522,Eleodora Tapia Lopez,Viudo,F,1983-09-05,Ninguna,cedula,cotizante,932170522
1,934170522,Belisaria Crespo Jerez,Union Libre,F,1990-12-23,Ninguna,cedula,cotizante,934170522
2,936170522,Gerardo Villanueva Pinto,Casado,M,1983-10-23,Ninguna,cedula,cotizante,936170522
3,937170522,Bienvenido Quintana Moreno,Viudo,M,1987-02-16,Ninguna,cedula,cotizante,937170522
4,941170522,Celestino Medrano Moreno,Union Libre,M,1990-01-24,Ninguna,cedula,cotizante,941170522


In [17]:
cotizantes.describe(include=["object", "bool"])cto

,numero_identificacion,nombre,estado_civil,sexo,tipo_discapacidad,tipo_documento,tipo_usuario,grupo_familiar
count,1100,1100,1100,1100,1100,1100,1100,1100
unique,1100,1100,5,2,4,1,1,1100
top,932170522,Eleodora Tapia Lopez,Viudo,M,Ninguna,cedula,cotizante,932170522
freq,1,1,236,563,1059,1100,1100,1


In [10]:
cotizantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   numero_identificacion  1100 non-null   object        
 1   nombre                 1100 non-null   object        
 2   estado_civil           1100 non-null   object        
 3   sexo                   1100 non-null   object        
 4   fecha_nacimiento       1100 non-null   datetime64[ns]
 5   tipo_discapacidad      1100 non-null   object        
 6   tipo_documento         1100 non-null   object        
 7   tipo_usuario           1100 non-null   object        
 8   grupo_familiar         1100 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 77.5+ KB


# Modificaciones Beneficiario

In [11]:
beneficiarios.drop(columns=['parentesco'], inplace=True)
beneficiarios.rename(columns={'tipo_identificacion':'tipo_documento','id_beneficiario':'numero_identificacion'}, inplace=True)
beneficiarios['tipo_usuario'] = "beneficiario"

In [12]:
beneficiario = beneficiarios.merge(cot_ben,left_on='numero_identificacion',right_on='beneficiario', how='left')
beneficiario.rename(columns={'cotizante': 'grupo_familiar'}, inplace=True)
beneficiario.drop(columns=['beneficiario'], inplace=True)
beneficiario.head()

,tipo_documento,numero_identificacion,nombre,fecha_nacimiento,sexo,estado_civil,tipo_discapacidad,tipo_usuario,grupo_familiar
0,Cedula,1705221112240,Deidamia Bernal Tello,1954-10-08,F,Soltero,Ninguna,beneficiario,1170522
1,Tarjeta Identidad,1705221115600,Nazaret Dueñas Dueñas,1997-10-16,F,Soltero,Ninguna,beneficiario,2170522
2,Cedula,1705221116830,Afra Acosta Heredia,1975-08-21,F,Union Libre,Ninguna,beneficiario,2170522
3,Cedula,1705221117960,Tomás Dueñas Delatorre,1967-07-24,M,Union Libre,Ninguna,beneficiario,2170522
4,Cedula,1705221123050,Celeste Díez Calderón,1949-08-05,F,Casado,Ninguna,beneficiario,5170522


# merging a dim_persona

In [13]:
from datetime import date

dim_persona = pd.concat([beneficiario,cotizantes])
dim_persona["saved"] = date.today()
dim_persona.reset_index(drop=True, inplace=True)
dim_persona.head()

,tipo_documento,numero_identificacion,nombre,fecha_nacimiento,sexo,estado_civil,tipo_discapacidad,tipo_usuario,grupo_familiar,saved
0,Cedula,1705221112240,Deidamia Bernal Tello,1954-10-08,F,Soltero,Ninguna,beneficiario,1170522,2024-09-17
1,Tarjeta Identidad,1705221115600,Nazaret Dueñas Dueñas,1997-10-16,F,Soltero,Ninguna,beneficiario,2170522,2024-09-17
2,Cedula,1705221116830,Afra Acosta Heredia,1975-08-21,F,Union Libre,Ninguna,beneficiario,2170522,2024-09-17
3,Cedula,1705221117960,Tomás Dueñas Delatorre,1967-07-24,M,Union Libre,Ninguna,beneficiario,2170522,2024-09-17
4,Cedula,1705221123050,Celeste Díez Calderón,1949-08-05,F,Casado,Ninguna,beneficiario,5170522,2024-09-17


In [14]:
dim_persona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4416 entries, 0 to 4415
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tipo_documento         4416 non-null   object        
 1   numero_identificacion  4416 non-null   object        
 2   nombre                 4416 non-null   object        
 3   fecha_nacimiento       4416 non-null   datetime64[ns]
 4   sexo                   4416 non-null   object        
 5   estado_civil           4416 non-null   object        
 6   tipo_discapacidad      4416 non-null   object        
 7   tipo_usuario           4416 non-null   object        
 8   grupo_familiar         4416 non-null   object        
 9   saved                  4416 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 345.1+ KB


# loading

In [15]:
dim_persona.to_sql('dim_persona',con=etl_conn,index_label='key_dim_persona',if_exists='replace')

416